In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection

def load_data(dropTipoVinho):
    data = pd.read_csv('../Data/Processed/wine/bigTableWines.csv', sep=";")
    if (dropTipoVinho):
        data = data.drop('Tipo Vinho', axis=1)
    return data

def get_target_and_classifier_and_classes_variables(data):
    target_variable = ['quality']
    classifier_variables = data.columns.drop(target_variable)
    classes=['baixa qualidade','alta qualidade']
    return target_variable, classifier_variables, classes
    
def validacao_cruzada(data, target_variable, classifier_variables):
    xData = data[classifier_variables].values
    yData = data[target_variable].values

    idx = yData > 5
    yData[idx] = 1
    yData[~idx]= 0

    # Dividir 20% dos dados para o conjunto out-of-sample de teste
    Xtrain, Xtest, Ytrain, Ytest = model_selection.train_test_split(xData, yData, test_size=0.2, random_state=0, stratify=yData)

    # Objeto de validacao cruzada
    cvfold = model_selection.StratifiedKFold(n_splits = 10, random_state = 0)
    
    return Xtrain, Xtest, Ytrain, Ytest, cvfold

def aplicar_modelo(model_pipe, Xtest):
    decision_threshold = 0.535
    Yhat = model_pipe.predict_proba(Xtest)[:,1]
    Ypred= Yhat >= decision_threshold
    print('Yhat.shape:', Yhat.shape)
    return Yhat, Ypred

def exibir_distribuicao(descricaoModelo, classes, Yhat, Ytest):
    for i in range(len(classes)):
        sns.distplot(Yhat[Ytest[:,0] == i], label=classes[i])
    plt.title(descricaoModelo)
    plt.ylabel('Densidade Estimada KDE')
    plt.xlabel('Probabilidade Vinho Alta Qualidade')
    plt.grid()
    plt.legend()
    